In [2]:
from FlagEmbedding import FlagModel

model = FlagModel("BAAI/bge-large-en", 
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages: "
    )

In [3]:
queries = ['conflicting', 'inexperience', 'assumption', 'appreciation', 'genetics', 'client relations']

In [4]:
import pandas as pd

consol_pre = pd.read_csv('./data/pre_consolidated.csv')
sep_pre = consol_pre[['ID'] + list(consol_pre.columns[14:-2])]
sep_pre_col_names = ['ID', 'Q14', 'Q15', 'Q16', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25']
sep_pre.columns = sep_pre_col_names

consol_post = pd.read_csv('./data/post_consolidated.csv')
sep_post = consol_post[['ID'] + list(consol_post.columns[14:-3])]
sep_post_col_names = ['ID', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']
sep_post.columns = sep_post_col_names

pre_fr = sep_pre[['ID', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25']]
post_fr = sep_post[['ID', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']]

In [12]:
from nltk import sent_tokenize

all_responses = []
for col in pre_fr.columns[1:]:
    pre_sent = pre_fr[col].apply(sent_tokenize)
    pre_sent = pre_sent.explode()
    for fr in list(pre_sent):
        all_responses.append(fr)

for col in post_fr.columns[1:]:
    post_sent = post_fr[col].apply(sent_tokenize)
    post_sent = post_sent.explode()
    for fr in list(post_sent):
        all_responses.append(fr)

len(all_responses)

253

In [13]:
q_embeddings = model.encode_queries(queries)
p_embeddings = model.encode(all_responses)

In [14]:
scores = q_embeddings @ p_embeddings.T

In [16]:
scores.shape

(6, 253)

In [21]:
scores[0].argsort()[::-1]

array([ 97,  18,  99, 132, 176, 164, 217,  91,  14, 221, 212, 175, 229,
       210,  81,   3,  15, 153, 209,  22,  10, 214,  89, 103,  82,  74,
         8, 124,  96, 219, 231,  70,  44, 136, 206, 213,  76, 101, 117,
       208, 222,  58, 104,   9, 137,  84,  67, 114,  66,  71, 100,  93,
         6, 220,  75,  85,  24,  29,  94, 139, 249, 190,  51,  83, 216,
       196, 226, 224,  79,  12,  80, 178,  88,  60, 129,  57, 228,  39,
       199, 105, 166, 133,   1,  65,   7, 111, 102,  95, 161, 113,   5,
       120, 223, 252, 211, 251, 189,  64,  77, 198,  68, 118, 119,  26,
       141,  78, 145, 225, 233, 154, 138, 148,  72, 246, 108, 131, 151,
        69,  54,  20, 165, 245, 243, 110, 204,  28, 184, 215,  27,  53,
       185, 134,  63, 116,  21, 128, 112,  32, 130,  92,  56,  98, 168,
        45, 240, 201, 163, 232, 160, 182, 158, 187, 140, 239,  73,  90,
       230, 152, 146, 107,  17, 244,   0, 135,  31, 156, 248,  16, 106,
       247, 194, 173,  42, 162,  52, 126, 121,   2, 237,  40,   

In [25]:
all_responses[132]

'I think the most predominate belief that was challenged is that not all breeders are the same.'